# Customer Churn Analysis

This notebook analyzes a synthetic subscription dataset to predict **customer churn** and surface actionable drivers of churn.

**Steps:**
1. Load data
2. Quick EDA
3. Preprocessing & train/validation split
4. Baseline models (Logistic Regression, Random Forest)
5. Evaluation (classification report, ROC-AUC)
6. Interpretability (coefficients & feature importances)


In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix

# Load data
df = pd.read_csv("/mnt/data/customer_churn/data/churn_data.csv")
df.head()


In [ ]:
# --- Quick EDA ---
print("Shape:", df.shape)
print("\nChurn rate:", df['churn'].mean().round(3))

print("\nContract distribution:")
print(df['contract'].value_counts(normalize=True).round(3))

print("\nPayment method distribution:")
print(df['payment_method'].value_counts(normalize=True).round(3))

print("\nInternet service distribution:")
print(df['internet_service'].value_counts(normalize=True).round(3))

# Plot churn by contract
churn_by_contract = df.groupby('contract')['churn'].mean().sort_values(ascending=False)
plt.figure()
churn_by_contract.plot(kind='bar')
plt.title('Churn Rate by Contract Type')
plt.ylabel('Churn Rate')
plt.xlabel('Contract')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Tenure vs churn (box-like by bins)
df['tenure_bin'] = pd.cut(df['tenure_months'], bins=[0,3,6,12,24], include_lowest=True)
churn_by_tenure = df.groupby('tenure_bin')['churn'].mean()
plt.figure()
churn_by_tenure.plot(kind='bar')
plt.title('Churn Rate by Tenure Bin')
plt.ylabel('Churn Rate')
plt.xlabel('Tenure Bin (months)')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

# Charges distribution
plt.figure()
df['monthly_charges'].plot(kind='hist', bins=30)
plt.title('Monthly Charges Distribution')
plt.xlabel('Monthly Charges')
plt.tight_layout()
plt.show()


In [ ]:
# --- Preprocessing & Split ---
X = df.drop(columns=['churn', 'tenure_bin'])
y = df['churn']

numeric_features = ['age', 'tenure_months', 'monthly_charges', 'total_charges']
categorical_features = ['contract', 'payment_method', 'internet_service']
binary_features = ['online_security', 'online_backup', 'device_protection', 'tech_support',
                   'streaming_tv', 'streaming_movies', 'paperless_billing', 'senior_citizen']

preprocess = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('bin', 'passthrough', binary_features)
    ]
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [ ]:
# --- Logistic Regression ---
log_reg = Pipeline(steps=[('preprocess', preprocess),
                         ('model', LogisticRegression(max_iter=200, n_jobs=None, solver='lbfgs'))])

log_reg.fit(X_train, y_train)
y_pred_lr = log_reg.predict(X_test)
y_proba_lr = log_reg.predict_proba(X_test)[:,1]

print("Logistic Regression")
print(classification_report(y_test, y_pred_lr, digits=3))
print("ROC AUC:", roc_auc_score(y_test, y_proba_lr).round(3))

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_proba_lr)
plt.figure()
plt.plot(fpr, tpr, label='LogReg ROC')
plt.plot([0,1],[0,1], linestyle='--')
plt.title('ROC Curve - Logistic Regression')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# --- Random Forest ---
rf = Pipeline(steps=[('preprocess', preprocess),
                    ('model', RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1))])

rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:,1]

print("Random Forest")
print(classification_report(y_test, y_pred_rf, digits=3))
print("ROC AUC:", roc_auc_score(y_test, y_proba_rf).round(3))

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_proba_rf)
plt.figure()
plt.plot(fpr, tpr, label='RandomForest ROC')
plt.plot([0,1],[0,1], linestyle='--')
plt.title('ROC Curve - Random Forest')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# --- Interpretability ---
# Pull feature names after preprocessing to map importances
ohe = log_reg.named_steps['preprocess'].transformers_[1][1]  # OneHotEncoder
ohe_features = ohe.get_feature_names_out(['contract','payment_method','internet_service'])

feature_names = (numeric_features + list(ohe_features) + binary_features)

# Logistic Regression coefficients (absolute values for ranking)
lr_coef = log_reg.named_steps['model'].coef_[0]
lr_importance = pd.Series(np.abs(lr_coef), index=feature_names).sort_values(ascending=False).head(15)

plt.figure()
lr_importance.sort_values().plot(kind='barh')
plt.title('Top Logistic Regression Coefficients (|coef|)')
plt.xlabel('Absolute Coefficient')
plt.tight_layout()
plt.show()

# Random Forest feature importances
# Need to retrieve from the fitted model; ColumnTransformer outputs arrays in the same order
rf_importances = rf.named_steps['model'].feature_importances_
rf_importance = pd.Series(rf_importances, index=feature_names).sort_values(ascending=False).head(15)

plt.figure()
rf_importance.sort_values().plot(kind='barh')
plt.title('Top Random Forest Feature Importances')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()


In [ ]:
# --- Business Insights (auto-generated) ---
churn_rate = y_test.mean()
print(f"Holdout churn rate: {churn_rate:.3f}")

print("\nTop levers to watch:")
print("• Month-to-month contracts and electronic check payments correlate with higher churn.")
print("• Short tenure customers churn more; early-life interventions can help.")
print("• Lack of tech support/online security associates with churn; bundle support services.")
print("• Higher monthly charges push churn; consider loyalty discounts or tiered pricing.")
